In [1]:
!date

Mon Jun 14 11:38:22 PDT 2021


# Notebook 3: Statistical analysis. 
This notebook was written by Victoria Jorgensen. Questions can be sent to jorgensen at caltech.edu. 

## Import packages. 

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

## Read in the data

In [2]:
adata = anndata.read_h5ad("../data/matrix_data/adata_analyzed.h5ad")
adata

AnnData object with n_obs × n_vars = 6231 × 33694
    obs: 'batch', 'sample_id', 'sample_group', 'lineage_id', 'name', 'cell_group', 'n_genes', 'n_counts', 'percent_mito', 'te_score', 'epi_score', 'hypo_score', 'epsc_score', 'lineage'
    var: 'gene_ids', 'feature_types', 'gene_name', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'cell_group_colors', 'hvg', 'name_colors', 'neighbors', 'pca', 'sample_group_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

### Make subsets of adata.

In [3]:
# Subset for EPI and ELCs
adata_epi = adata[adata.obs['lineage']== 'epi'] 
adata_epi = adata[(adata.obs['lineage_id']== 'Epiblast') | 
                  (adata.obs['lineage_id']== 'D5 ELCs') |  
                  (adata.obs['lineage_id']== 'D6 ELCs')] 

# Subset for HYPO and HLCs
adata_hypo = adata[adata.obs['lineage']== 'hypo']
adata_hypo = adata[(adata.obs['lineage_id']== 'Hypoblast') | 
                  (adata.obs['lineage_id']== 'D5 HLCs') |  
                  (adata.obs['lineage_id']== 'D6 HLCs')] 

# Subset for TE and TLCs
adata_te = adata[adata.obs['lineage']== 'te']
adata_te = adata[(adata.obs['lineage_id']== 'Trophectoderm') | 
                  (adata.obs['lineage_id']== 'D5 TLCs') |  
                  (adata.obs['lineage_id']== 'D6 TLCs')] 

### Make gene list for each lineage

In [5]:
# Reads in a list of genes for each lineage from from supplementary table 12 of Liu et al.
df_lineage = pd.read_csv('../data/supp12.csv')
epi_markers = df_lineage.loc[df_lineage["type"]=="ALL-EPI"]["geneName"].values
hypo_markers = df_lineage.loc[df_lineage["type"]=="ALL-PE"]["geneName"].values
te_markers = df_lineage.loc[df_lineage["type"]=="ALL-TE"]["geneName"].values

# Makes sure that all genes in lineage gene lists are also present in the data set. 
epi_markers_present = []
for marker in epi_markers:
    if marker in adata.var["gene_name"]:
        epi_markers_present.append(marker)
        
hypo_markers_present = []
for marker in hypo_markers:
    if marker in adata.var["gene_name"]:
        hypo_markers_present.append(marker)
        
te_markers_present = []
for marker in te_markers:
    if marker in adata.var["gene_name"]:
        te_markers_present.append(marker)


___
## Mean, Std, and max expression values for HYPO, EPI, TE
___

### EPIBLAST

EPI mean

In [13]:
# Mean expression for genes in ELCs/Epiblast
epi_res_mean = pd.DataFrame(columns=adata_epi.var_names, index=adata_epi.obs['lineage_id'].cat.categories)                                                                                                 
for clust in adata_epi.obs.lineage_id.cat.categories: 
    epi_res_mean.loc[clust] = adata_epi[adata_epi.obs['lineage_id'].isin([clust]),:].X.mean(0)
    
# Dataframe with EPI related genes only.  
df_epi_res_mean = pd.DataFrame()
for i, marker in enumerate(epi_markers_present):
    df_epi_res_mean[marker] = epi_res_mean[marker]   
    
# # Transform matrix and sort values    
df_epi_res_mean = df_epi_res_mean.T
df_epi_res_mean = df_epi_res_mean.sort_values("Epiblast")
df_epi_res_mean

EPI standard deviation

In [14]:
# Standard deviation for ELCs/Epiblast     
epi_res_std = pd.DataFrame(columns=adata_epi.var_names, index=adata_epi.obs['lineage_id'].cat.categories)                                                                                                 

for clust in adata_epi.obs.lineage_id.cat.categories: 
    epi_res_std.loc[clust] = adata_epi[adata_epi.obs['lineage_id'].isin([clust]),:].X.std(0)

# Dataframe with EPI related genes only.   
df_epi_res_std = pd.DataFrame()
for i, marker in enumerate(epi_markers_present):
    df_epi_res_std[marker] = epi_res_std[marker]
    
# # Transform matrix and sort values
df_epi_res_std = df_epi_res_std.T
df_epi_res_std = df_epi_res_std.sort_values("Epiblast")
df_epi_res_std

,D5 ELCs,D6 ELCs,Epiblast
PLA2G4F,9.31323e-09,8.3819e-09,2.79397e-09
BTLA,9.31323e-09,8.3819e-09,2.79397e-09
ZYG11A,5.96046e-08,2.42144e-08,5.58794e-09
LINC01108,3.91155e-08,2.79397e-08,5.58794e-09
MUC4,4.84288e-08,1.86265e-08,7.45058e-09
...,...,...,...
SUSD2,0.732946,1.08275,4.4067
DND1,1.14189,1.26228,4.47493
GDF3,0.661429,0.673484,4.49596
UTF1,7.45058e-08,0.174632,4.78246


### HYPOBLAST

HYPO mean

In [15]:
# Mean expression for genes in HLCs/Hypoblast  
res_hypo_mean = pd.DataFrame(columns=adata_hypo.var_names, index=adata_hypo.obs['lineage_id'].cat.categories)                                                                                                 

for clust in adata_hypo.obs.lineage_id.cat.categories: 
    res_hypo_mean.loc[clust] = adata_hypo[adata_hypo.obs['lineage_id'].isin([clust]),:].X.mean(0)

# Dataframe with HYPO related genes only.      
df_hypo_res_mean = pd.DataFrame()
for i, marker in enumerate(hypo_markers_present):
    df_hypo_res_mean[marker] = res_hypo_mean[marker]

# # Transform matrix and sort values
df_hypo_res_mean = df_hypo_res_mean.T
df_hypo_res_mean = df_hypo_res_mean.sort_values("Hypoblast")
df_hypo_res_mean

,D5 HLCs,D6 HLCs,Hypoblast
TMEM88,0.756147,0.112712,-1.3698
COL4A2,-0.123206,0.7436,-0.995935
COL4A1,-0.224466,0.788984,-0.927994
FN1,-0.421687,0.843742,-0.913916
RGS5,1.00669,0.0595427,-0.904628
...,...,...,...
DENND2C,0.248629,0.00156986,3.94788
FOXA2,-0.0230933,-0.0489156,3.95292
LINC00261,-0.0513399,-0.0430712,3.9692
HNF1B,-0.0349656,-0.0349656,3.97902


HYPO standard deviation

In [16]:
# Standard deviation in HLCs/Hypoblast     
res_hypo_std = pd.DataFrame(columns=adata_hypo.var_names, index=adata_hypo.obs['lineage_id'].cat.categories)                                                                                                 

for clust in adata_hypo.obs.lineage_id.cat.categories: 
    res_hypo_std.loc[clust] = adata_hypo[adata_hypo.obs['lineage_id'].isin([clust]),:].X.std(0)

# Dataframe with HYPO related genes only. 
df_hypo_res_std = pd.DataFrame()
for i, marker in enumerate(hypo_markers_present):
    df_hypo_res_std[marker] = res_hypo_std[marker]   
    
# # Transform matrix and sort values
df_hypo_res_std = df_hypo_res_std.T
df_hypo_res_std = df_hypo_res_std.sort_values("Hypoblast")
df_hypo_res_std

,D5 HLCs,D6 HLCs,Hypoblast
FGG,8.00937e-08,0.737306,0
CELA3A,0,0,0
CALCR,5.58794e-08,3.72529e-08,0
TMEM88,1.01561,0.847391,0
GCKR,0,0,0
...,...,...,...
APOA1,0.631223,1.61868,4.63922
HNF1B,1.49012e-07,1.71363e-07,4.91611
LINC00261,2.68221e-07,0.19392,4.92413
FOXA2,0.745011,0.48987,4.93742


### TROPHOBLAST

TE mean

In [17]:
# Mean expression in TLCs/Trophectoderm     
res_te_mean = pd.DataFrame(columns=adata_te.var_names, index=adata_te.obs['lineage_id'].cat.categories)                                                                                                 

for clust in adata_te.obs.lineage_id.cat.categories: 
    res_te_mean.loc[clust] = adata_te[adata_te.obs['lineage_id'].isin([clust]),:].X.mean(0)
    
# Dataframe with TE related genes only. 
df_te_res_mean = pd.DataFrame()
for i, marker in enumerate(te_markers_present):
    df_te_res_mean[marker] = res_te_mean[marker] 
    
# # Transform matrix and sort values
df_te_res_mean = df_te_res_mean.T
df_te_res_mean = df_te_res_mean.sort_values("Trophectoderm")
df_te_res_mean

,D5 TLCs,D6 TLCs,Trophectoderm
ACTN1,-0.328309,1.03118,-0.590141
HSPB11,0.143427,0.209658,-0.494335
CAST,0.274772,0.187926,-0.454218
PALLD,0.212631,0.548377,-0.410884
ANXA6,-0.0854063,0.211437,-0.38422
...,...,...,...
GATA2,-0.0750121,-0.160698,2.11157
GATA3,0.843628,-0.077936,2.16422
FOLR1,-0.0728055,-0.15972,2.19455
S100A6,-0.237083,-0.223001,2.40751


TE standard deviation

In [18]:
# Standard deviation for TLCs/Trophectoderm   
res_te_std = pd.DataFrame(columns=adata_te.var_names, index=adata_te.obs['lineage_id'].cat.categories)                                                                                                 

for clust in adata_te.obs.lineage_id.cat.categories: 
    res_te_std.loc[clust] = adata_te[adata_te.obs['lineage_id'].isin([clust]),:].X.std(0)

# Dataframe with TE related genes only. 
df_te_res_std = pd.DataFrame()
for i, marker in enumerate(te_markers_present):
    df_te_res_std[marker] = res_te_std[marker]
    
# # Transform matrix and sort values
df_te_res_std = df_te_res_std.T
df_te_res_std = df_te_res_std.sort_values("Trophectoderm")
df_te_res_std

,D5 TLCs,D6 TLCs,Trophectoderm
KRT18,0.867536,0.550452,0.484772
KRT8,0.70714,0.378119,0.692105
NIM1K,2.23517e-08,2.6077e-08,0.897014
CYP26A1,1.1085,0.312652,0.929208
ACTN1,0.888744,0.716178,0.939966
...,...,...,...
CLDN3,1.14351,1.18903,3.04888
SLC12A3,2.23517e-08,1.49012e-08,3.05401
ATP6V0A4,3.72529e-08,1.49012e-08,3.05536
SLC7A4,1.36622,6.70552e-08,3.08376


In [22]:
# Concatenate the mean and std dataframes for each lineage. 
df_epi_data = pd.concat([df_epi_res_mean, df_epi_res_std])
df_hypo_data = pd.concat([df_hypo_res_mean, df_hypo_res_std])
df_te_data = pd.concat([df_te_res_mean, df_te_res_std])

# # Save as .csv
df_epi_data.to_csv("../data/matrix_data/stats/epi_mean_std.csv")
df_hypo_data.to_csv("../data/matrix_data/stats/hypo_mean_std.csv")
df_te_data.to_csv("../data/matrix_data/stats/te_mean_std.csv")

In [25]:
# Shows counts for all lineages.
df_counts = pd.DataFrame((adata.obs.lineage_id.value_counts())).T
df_counts = df_counts[["D5 ELCs", "D6 ELCs","Epiblast", 
                       "D5 HLCs", "D6 HLCs", "Hypoblast", 
                       "D5 TLCs", "D6 TLCs","Trophectoderm", 
                       "Undefined"]]
df_counts.to_csv("../data/matrix_data/stats/lineage_counts.csv")

In [24]:
%load_ext watermark
%watermark -v -p numpy,pandas,scanpy,anndata,jupyterlab,matplotlib

CPython 3.7.10
IPython 7.22.0

numpy 1.19.4
pandas 1.1.4
scanpy 1.7.2
anndata 0.7.6
jupyterlab 3.0.11
matplotlib 3.3.4
